In [1]:
import pandas as pd
import numpy as np
import pm4py
import os
from sklearn.neighbors import KernelDensity
import random
from collections import Counter
import copy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def csv_to_xes(file, add_startend, id_column, act_column, start_time_column, end_time_column, resource_column):
    import_file = f'{file}.csv' #
    df = pd.read_csv(import_file)
#     print(df)
    id_column = id_column
    act_column = act_column
    start_time_column = start_time_column
    end_time_column = end_time_column
    resource_column = resource_column
    state_column = 'lifecycle:transition'
    time_column = 'time:timestamp'
    df[resource_column].fillna('no_resource', inplace=True)
    df.to_csv(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/{file}.csv', index=False)
    if add_startend:
        # pre-processing
        mydata = pd.DataFrame()
        for key, group in df.groupby(id_column):
            group = add_start_end(df, group, act_column, resource_column, start_time_column, end_time_column)   # add start event in the beginning and end in the last 
            mydata = pd.concat([mydata, group], ignore_index=True)
        df = mydata

    
    df_start = df[[id_column, act_column, start_time_column, resource_column]].rename(columns={start_time_column:time_column})
    df_start[state_column] = 'start'
    df_start

    df_end = df[[id_column, act_column, end_time_column, resource_column]].rename(columns={end_time_column:time_column})
    df_end[state_column] = 'complete'

    df_start_end = pd.concat([df_start, df_end])
    df_start_end['index'] = df_start_end.index
    df_start_end[time_column] = pd.to_datetime(df_start_end[time_column])
    new = pd.DataFrame()
    for key, group in df_start_end.groupby(id_column):
        temp = group.sort_values(by=[time_column, 'index'], ascending=[True, True]).reset_index(drop=True)
        new = pd.concat([new,temp],ignore_index = True)
    del new['index']
    df = new
    
    
#     print(new.columns)
    df['concept:name'] = df[act_column].astype('str')
    df[id_column] = df[id_column].astype('str')
    print(df)
    df = df[[id_column, 'concept:name', state_column, resource_column, time_column, act_column]]
#     df = df[[id_column, act_column,  state_column, resource_column, time_column]]
    df = df.rename(columns = { resource_column: 'user', act_column: 'task'})
    
    pm4py.write.write_xes(df, f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/{file}.xes', case_id_key=id_column)

In [ ]:
csv_to_xes('train_ConsultaDataMining201618', add_startend = False, id_column='caseid', act_column='task', start_time_column='start_timestamp', end_time_column='end_timestamp', resource_column='user')

In [2]:
# info of train_data
if_csv = False #
import_file = f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/train_ConsultaDataMining201618.xes' #
id_column = 'caseid'
act_column = 'concept:name'
time_column = 'time:timestamp'
resource_column = 'user'
state_column = 'lifecycle:transition'

dirStr, ext = os.path.splitext(import_file)
file_name = dirStr.split("/")[-1]


data = pm4py.read.read_xes(import_file)

df = copy.deepcopy(pd.DataFrame(data,columns=[id_column, act_column, time_column, resource_column, state_column]))

parsing log, completed traces ::   0%|          | 0/763 [00:00<?, ?it/s]

In [3]:
df['waiting_time'] = None
df['process_time'] = None
df['last_complete_event'] = None
df['preceding_evts'] = None
df['paired_event'] = None
df['next'] = None
df['index'] = df.index
for key, group in df.groupby(id_column):
    flag = 0
    i = list(group.index)[0]
    preceding_evt = []
    not_complete_evt_idx = []
    j = 0
    df['waiting_time'].loc[i] = 0
    not_complete_evt_idx.append(i)
    i+=1
    while j<len(group)-1:
        j+=1
        cur_act = df.loc[i]
        if cur_act[state_column] == 'complete':
            flag = 0
            preceding_evt.append(cur_act.name)
            last_complete_evt_idx = i
            
            for each_idx in not_complete_evt_idx:
                to_pair = df.loc[each_idx]
                if (cur_act[act_column] == to_pair[act_column]) \
                and (cur_act[resource_column] == to_pair[resource_column]):
                    df['paired_event'].loc[i] = each_idx
                    not_complete_evt_idx.remove(each_idx)
                    df['process_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[each_idx]).total_seconds()
                    df['last_complete_event'].loc[i] = df['last_complete_event'].loc[each_idx]
                    break
            
        else:
            if flag == 1:
                df['preceding_evts'].loc[i] = df['preceding_evts'].loc[i-1]
            else:
                df['preceding_evts'].loc[i] = preceding_evt#tuple(sorted, reverse=False))
            for each in df['preceding_evts'].loc[i]:
                if df['next'].loc[each] == None:
                    df['next'].loc[each] = [i]
                else:
                    temp = df['next'].loc[each]
                    temp.append(i)
                    df['next'].loc[each] = temp
            flag = 1
            preceding_evt = []
            not_complete_evt_idx.append(i)
            
            df['last_complete_event'].loc[i] = last_complete_evt_idx
            df['waiting_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[last_complete_evt_idx]).total_seconds()
        i+=1

In [4]:
def fillna_parallel_data(bb):
    i = bb.index[0]
    j = 0 
    while j < len(bb):
        if bb[state_column].loc[i] == 'complete':
            paired = int(bb['paired_event'].loc[i])
            bb['process_time'].loc[paired] = bb['process_time'].loc[i] 
            bb['paired_event'].loc[paired] = i
            bb['next'].loc[paired] = bb['next'].loc[i]
            bb['waiting_time'].loc[i] = bb['waiting_time'].loc[paired] 
            bb['preceding_evts'].loc[i] = bb['preceding_evts'].loc[paired]
            bb['next'].loc[paired] = bb['next'].loc[i] 
        i+=1
        j+=1
fillna_parallel_data(df)


In [23]:
df

,caseid,concept:name,time:timestamp,user,lifecycle:transition,waiting_time,process_time,last_complete_event,preceding_evts,paired_event,next,index
0,81102,Start,2016-02-01 18:53:46+00:00,Start,start,0,0,None,None,1,[2],0
2,81102,Traer informacion estudiante - banner,2016-02-01 18:53:46+00:00,6426,start,0,0,1,[1],3,[4],2
4,81102,Radicar Solicitud Homologacion,2016-02-01 18:53:50+00:00,6426,start,4,966,3,[3],5,[6],4
6,81102,Validar solicitud,2016-02-03 19:47:36+00:00,22475,start,175060,119,5,[5],7,[8],6
8,81102,Homologacion por grupo de cursos,2016-02-03 22:59:19+00:00,1343,start,11384,2959,7,[7],9,[10],8
...,...,...,...,...,...,...,...,...,...,...,...,...
10966,129262,Traer informacion estudiante - banner,2016-06-28 15:08:14+00:00,19692,start,0,0,10965,[10965],10967,[10968],10966
10968,129262,Radicar Solicitud Homologacion,2016-06-28 15:08:15+00:00,19692,start,1,0,10967,[10967],10969,[10970],10968
10970,129262,Validar solicitud,2016-06-28 15:10:32+00:00,29277,start,137,43,10969,[10969],10971,[10972],10970
10972,129262,Homologacion por grupo de cursos,2016-06-28 16:43:19+00:00,1007,start,5524,246,10971,[10971],10973,[10974],10972


In [34]:
df = df[df[state_column]=='start']

# df = df.rename(columns = {'concept:name':'activity'})
file = df[[id_column,act_column, resource_column, 'waiting_time', 'process_time']]
file = file[file[act_column] != 'Start']
file = file[file[act_column] != 'End']
file = file.reset_index(drop=True)

file[act_column] = file[act_column].map(lambda x: '_'.join(x.split()))
file[resource_column] = file[resource_column].map(lambda x: '_'.join(x.split()))

file

,caseid,concept:name,user,waiting_time,process_time
0,81102,Traer_informacion_estudiante_-_banner,6426,0,0
1,81102,Radicar_Solicitud_Homologacion,6426,4,966
2,81102,Validar_solicitud,22475,175060,119
3,81102,Homologacion_por_grupo_de_cursos,1343,11384,2959
4,81201,Traer_informacion_estudiante_-_banner,8916,0,0
...,...,...,...,...,...
3957,129254,Homologacion_por_grupo_de_cursos,1007,6567,261
3958,129262,Traer_informacion_estudiante_-_banner,19692,0,0
3959,129262,Radicar_Solicitud_Homologacion,19692,1,0
3960,129262,Validar_solicitud,29277,137,43


In [35]:
import math
file['waiting_time'] = file['waiting_time'].map(lambda x: math.log(x+1))
file['process_time'] = file['process_time'].map(lambda x: math.log(x+1))
file

,caseid,concept:name,user,waiting_time,process_time
0,81102,Traer_informacion_estudiante_-_banner,6426,0.000000,0.000000
1,81102,Radicar_Solicitud_Homologacion,6426,1.609438,6.874198
2,81102,Validar_solicitud,22475,12.072890,4.787492
3,81102,Homologacion_por_grupo_de_cursos,1343,9.340052,7.992945
4,81201,Traer_informacion_estudiante_-_banner,8916,0.000000,0.000000
...,...,...,...,...,...
3957,129254,Homologacion_por_grupo_de_cursos,1007,8.789965,5.568345
3958,129262,Traer_informacion_estudiante_-_banner,19692,0.000000,0.000000
3959,129262,Radicar_Solicitud_Homologacion,19692,0.693147,0.000000
3960,129262,Validar_solicitud,29277,4.927254,3.784190


In [36]:
from collections import Counter
from sklearn import preprocessing
new_data = pd.DataFrame()
def get_info(df, new_df, column_name, column_type):
    if column_name == id_column:
        new_df[column_name] = df[column_name]
    else:  
        if column_type == 'categorical':
            df[column_name].fillna(f'NO_{column_name}',inplace=True)
            dict_ = {}
            a = Counter(df[column_name])     
            dict_['name'] = column_name
            dict_['size'] = len(a.keys())
            dict_['type'] = column_type
            enc=preprocessing.LabelEncoder()
            enc=enc.fit(list(a.keys()))#训练LabelEncoder,将电脑，手表，手机编码为0,1,2
            data=enc.transform(df[column_name])#使用训练好的LabelEncoder对原数据进行编码，也叫归一化
            dict_['i2s'] = list(enc.classes_)
            new_df[column_name] = data
            return dict_
        else:
            dict_ = {}
            dict_['max'] = max(df[column_name])
            dict_['min'] = min(df[column_name])
            dict_['name'] = column_name
            dict_['type'] = column_type
            new_df[column_name] = df[column_name]
            return dict_


In [74]:
new_data = pd.DataFrame()
a = [act_column, resource_column]
b = ['waiting_time', 'process_time']
c = [act_column, resource_column]

json_input = {}
get_info(file, new_data, id_column, 'categorical')
json_input['attention'] = [get_info(file, new_data, each, 'categorical') for each in c]
json_input['columns'] = [get_info(file, new_data, each, 'categorical') for each in a]
json_input['columns'].extend([get_info(file, new_data, each, 'continuous') for each in b])
json_input['problem_type'] = 'no'

In [50]:
for i in range(1):
    print(i)

0


In [12]:
# new_data['prev_acts'] = None
# new_data['prev_res'] = None
# new_data['curr_act'] = None
# data_add_prev = pd.DataFrame()
# for key, group in new_data.groupby(id_column):
#     for i in range(len(group)):
#         prev_acts = list(group[act_column].iloc[0:i].astype('str'))
#         prev_acts_str = ' '.join(prev_acts)
#         group['prev_acts'].iloc[i] = prev_acts_str
        
#         prev_res = list(group[resource_column].iloc[0:i].astype('str'))
#         prev_res_str = ' '.join(prev_res)
#         group['prev_res'].iloc[i] = prev_res_str
        
#         group['curr_act'].iloc[i] = group[act_column].iloc[i].astype('str')
#     data_add_prev = pd.concat([data_add_prev, group])

In [75]:
new_data['prev_acts'] = None
new_data['prev_res'] = None
new_data['curr_act'] = None
data_add_prev = pd.DataFrame()
for key, group in new_data.groupby(id_column):
    for i in range(len(group)):
#         if i != 0:
        prev_acts = list(group[act_column].iloc[0:i])
#         prev_acts_str = ' '.join(prev_acts)
        group['prev_acts'].iloc[i] = prev_acts

        prev_res = list(group[resource_column].iloc[0:i])
#         prev_res_str = ' '.join(prev_res)
        group['prev_res'].iloc[i] = prev_res
        #print(group)
        group['curr_act'].iloc[i] = group[act_column].iloc[i]
    data_add_prev = pd.concat([data_add_prev, group])

In [76]:
data_add_prev['curr_act'] = data_add_prev['curr_act'].map(lambda x: [x])
data_add_prev

,caseid,concept:name,user,waiting_time,process_time,prev_acts,prev_res,curr_act
1347,100952,9,295,0.000000,0.000000,[],[],[9]
1348,100952,6,295,0.693147,8.245122,[9],[295],[6]
1349,100952,13,105,14.198515,8.550628,"[9, 6]","[295, 295]",[13]
1350,100952,4,99,14.061679,9.757363,"[9, 6, 13]","[295, 295, 105]",[4]
1351,100952,8,105,14.061679,8.581857,"[9, 6, 13, 4]","[295, 295, 105, 99]",[8]
...,...,...,...,...,...,...,...,...
1342,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11]","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11]
1343,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11]","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11]
1344,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11]
1345,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11]


In [78]:
def padding(list_, pad_number, length):
    len_list = len(list_) 
    if len_list < length:
        list_ = list_+ (length - len_list)*[pad_number]
    return list_

def padding_key(list_,act_pad):
    return [True if each == act_pad else False for each in list_]

In [79]:
len_act = max(data_add_prev['prev_acts'].map(lambda x: len(x)))
len_res = max(data_add_prev['prev_res'].map(lambda x: len(x)))
act_pad = json_input['attention'][0]['size']
res_pad = json_input['attention'][1]['size']



data_add_prev['prev_acts'] = data_add_prev['prev_acts'].map(lambda x: padding(x,act_pad,len_act))
data_add_prev['prev_acts_key_padding'] = data_add_prev['prev_acts'].map(lambda x: padding_key(x,act_pad))
data_add_prev['prev_res'] = data_add_prev['prev_res'].map(lambda x: padding(x,res_pad,len_res))
data_add_prev['prev_res_key_padding'] = data_add_prev['prev_res'].map(lambda x: padding_key(x,res_pad))
json_input['attention'][0]['len'] = len_act
json_input['attention'][1]['len'] = len_res

In [80]:

data_add_prev

,caseid,concept:name,user,waiting_time,process_time,prev_acts,prev_res,curr_act,prev_acts_key_padding,prev_res_key_padding
1347,100952,9,295,0.000000,0.000000,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1...","[503, 503, 503, 503, 503, 503, 503, 503, 503, ...",[9],"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
1348,100952,6,295,0.693147,8.245122,"[9, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[295, 503, 503, 503, 503, 503, 503, 503, 503, ...",[6],"[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr..."
1349,100952,13,105,14.198515,8.550628,"[9, 6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,...","[295, 295, 503, 503, 503, 503, 503, 503, 503, ...",[13],"[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T..."
1350,100952,4,99,14.061679,9.757363,"[9, 6, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16,...","[295, 295, 105, 503, 503, 503, 503, 503, 503, ...",[4],"[False, False, False, True, True, True, True, ...","[False, False, False, True, True, True, True, ..."
1351,100952,8,105,14.061679,8.581857,"[9, 6, 13, 4, 16, 16, 16, 16, 16, 16, 16, 16, ...","[295, 295, 105, 99, 503, 503, 503, 503, 503, 5...",[8],"[False, False, False, False, True, True, True,...","[False, False, False, False, True, True, True,..."
...,...,...,...,...,...,...,...,...,...,...
1342,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 16,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11],"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."
1343,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11],"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."
1344,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11],"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."
1345,99801,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...",[11],"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."


In [81]:
data = data_add_prev[[act_column, resource_column, 'waiting_time', 'process_time', 'prev_acts', 'prev_res','prev_acts_key_padding','prev_res_key_padding', 'curr_act']]
data = data.reset_index(drop = True)
data

,concept:name,user,waiting_time,process_time,prev_acts,prev_res,prev_acts_key_padding,prev_res_key_padding,curr_act
0,9,295,0.000000,0.000000,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1...","[503, 503, 503, 503, 503, 503, 503, 503, 503, ...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...",[9]
1,6,295,0.693147,8.245122,"[9, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[295, 503, 503, 503, 503, 503, 503, 503, 503, ...","[False, True, True, True, True, True, True, Tr...","[False, True, True, True, True, True, True, Tr...",[6]
2,13,105,14.198515,8.550628,"[9, 6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,...","[295, 295, 503, 503, 503, 503, 503, 503, 503, ...","[False, False, True, True, True, True, True, T...","[False, False, True, True, True, True, True, T...",[13]
3,4,99,14.061679,9.757363,"[9, 6, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16,...","[295, 295, 105, 503, 503, 503, 503, 503, 503, ...","[False, False, False, True, True, True, True, ...","[False, False, False, True, True, True, True, ...",[4]
4,8,105,14.061679,8.581857,"[9, 6, 13, 4, 16, 16, 16, 16, 16, 16, 16, 16, ...","[295, 295, 105, 99, 503, 503, 503, 503, 503, 5...","[False, False, False, False, True, True, True,...","[False, False, False, False, True, True, True,...",[8]
...,...,...,...,...,...,...,...,...,...
3957,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 16,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",[11]
3958,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",[11]
3959,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",[11]
3960,11,237,0.000000,0.000000,"[9, 6, 13, 4, 7, 8, 3, 14, 12, 15, 10, 11, 11,...","[445, 445, 332, 237, 209, 332, 346, 332, 332, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",[11]


In [82]:
num_train = round(len(data)*0.8)
num_train

3170

In [103]:
[100,10].index(10)

1

In [83]:
import json
with open("res_time_prev.json","w") as f:
    json.dump(json_input,f)

# with open('test.json') as json_file:
#     a = json.load(json_file)

train = np.array(data.iloc[:num_train])
train_1 = train[:,0:-5].astype('float32')
train_attention = train[:,-5:]
train

test = np.array(data.iloc[num_train:])
test_1 = test[:,0:-5].astype('float32')
test_attention = test[:,-5:]
test.shape
train_1


array_dict = {'train':train_1, 'test':test_1, 'train_attention': train_attention, 'test_attention' :test_attention}
np.savez('res_time_prev.npz', **array_dict)




In [84]:
a = np.load('res_time_prev.npz',allow_pickle=True)
a['train'].shape

(3170, 4)

In [81]:
!pip install torchtext==0.6.0


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [151]:
from torchtext.vocab import Vocab,build_vocab_from_iterator

from collections import Counter, OrderedDict
import torch


In [92]:

import pandas as pd
import numpy as np
import torch
gen = pd.read_csv('new_technique_train_ConsultaDataMining201618.csv')

gen['act'] = gen['act'].map(lambda x: '_'.join(x.split()))
gen_act = gen['act']
gen_act = gen_act.map(lambda x:json_input['columns'][0]['i2s'].index(x) if x in json_input['columns'][0]['i2s'] else x)


gen_res = []
gen_act = list(gen_act)
acts_prev = []
res_prev = []
for i in range(len(gen_act)):
    if gen_act[i] == 'Start':
        start_flag=i
        gen_res.append('Start')
    elif gen_act[i] == 'End':
        gen_res.append('End')
        # pass
    else:
        cur_act = [gen_act[i]]
        if i == start_flag+1:
            acts_prev = [23]
            res_prev = [13]

            acts_padding = [True]
            res_padding = [True]
        elif i == start_flag+2:
            acts_prev = []
            res_prev = []
            acts_prev.append(gen_act[i-1])
            res_prev.append(gen_res[i-1])

            acts_padding = [False for each in acts_prev]
            res_padding = [False for each in res_prev]
        else:
            acts_prev.append(gen_act[i - 1])
            res_prev.append(gen_res[i - 1])

            acts_padding = [False for each in acts_prev]
            res_padding = [False for each in res_prev]
        
        attention_tensor_list = [torch.tensor(acts_prev), torch.tensor(res_prev),
                                             torch.tensor(acts_padding),torch.tensor(res_padding),torch.tensor(cur_act) ]
        print(attention_tensor_list)
        gen_res.append(0)


[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([5])]
[tensor([23]), tensor([13]), tensor([True]), tensor([True]), tensor([9])]
[tensor([9]), tensor([0]), tensor([False]), tensor([False]), tensor([6])]
[tensor([9, 6]), tensor([0, 0]), tensor([False, False]), tensor([False, False]), tensor([1])]
[tensor([9, 6, 1]), tensor([0, 0

In [179]:
act_vocab = build_vocab(column='activity', min_freq=1)
act_vocab.get_stoi()

{'<bos>': 2,
 'Pending_Request_for_Reservation_Closure': 6,
 '<unk>': 0,
 'Request_created': 4,
 '<eos>': 3,
 '<pad>': 1,
 'Service_closure_Request_with_BO_responsibility': 5,
 'Pending_Request_for_acquittance_of_heirs': 12,
 'Request_completed_with_account_closure': 7,
 'Pending_Liquidation_Request': 8,
 'Service_closure_Request_with_network_responsibility': 9,
 'Authorization_Requested': 10,
 'Request_deleted': 14,
 'Evaluating_Request_(NO_registered_letter)': 11,
 'Network_Adjustment_Requested': 13,
 'Back-Office_Adjustment_Requested': 15,
 'Pending_Request_for_Network_Information': 16,
 'Evaluating_Request_(WITH_registered_letter)': 17}

In [177]:
class LoadActResDataset():
    def __init__(self, batch_size=2, min_freq=1):
        self.act_vocab = build_vocab(column='activity', min_freq=min_freq)
        self.res_vocab = build_vocab(column='user', min_freq=min_freq)
        self.specials = ['<unk>', '<pad>', '<bos>', '<eos>']
        self.PAD_IDX = self.act_vocab['<pad>']
        self.BOS_IDX = self.act_vocab['<bos>']
        self.EOS_IDX = self.act_vocab['<eos>']
        self.batch_size = batch_size
        
    def data_process(self):
        """
        将每一句话中的每一个词根据字典转换成索引的形式
        :param filepaths:
        :return:
        """
        raw_act_iter = data_add_prev['prev_acts']
        raw_res_iter = data_add_prev['prev_res']
        raw_curr_act_iter = data_add_prev['curr_act']
        data = []
        for (raw_act, raw_res, curr_act) in zip(raw_act_iter, raw_res_iter, raw_curr_act_iter):
            act_tensor_ = torch.tensor([self.act_vocab[token] for token in
                                       raw_act], dtype=torch.long)
            res_tensor_ = torch.tensor([self.res_vocab[token] for token in
                                       raw_res], dtype=torch.long)
            curr_act_tensor_ = torch.tensor([self.act_vocab[token] for token in
                                       curr_act], dtype=torch.long)
            data.append((act_tensor_, res_tensor_, curr_act_tensor_))
        return data



In [178]:
d = LoadActResDataset()
d.data_process()

[(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([4])),
 (tensor([4]), tensor([258]), tensor([9])),
 (tensor([4, 9]), tensor([258,   5]), tensor([10])),
 (tensor([ 4,  9, 10]), tensor([258,   5, 204]), tensor([5])),
 (tensor([ 4,  9, 10,  5]), tensor([258,   5, 204,   4]), tensor([6])),
 (tensor([ 4,  9, 10,  5,  6]),
  tensor([258,   5, 204,   4,   4]),
  tensor([8])),
 (tensor([ 4,  9, 10,  5,  6,  8]),
  tensor([258,   5, 204,   4,   4,   4]),
  tensor([7])),
 (tensor([], dtype=torch.int64), tensor([], dtype=torch.int64), tensor([4])),
 (tensor([4]), tensor([151]), tensor([11])),
 (tensor([ 4, 11]), tensor([151,  64]), tensor([9])),
 (tensor([ 4, 11,  9]), tensor([151,  64,  64]), tensor([5])),
 (tensor([ 4, 11,  9,  5]), tensor([151,  64,  64,   4]), tensor([6])),
 (tensor([ 4, 11,  9,  5,  6]),
  tensor([151,  64,  64,   4,   4]),
  tensor([8])),
 (tensor([ 4, 11,  9,  5,  6,  8]),
  tensor([151,  64,  64,   4,   4,   4]),
  tensor([7])),
 (tensor([], dtype=t

In [118]:
counter = Counter()
counter.update([each_ for each in data_add_prev['prev_acts'] for each_ in each])
counter

Counter({'Request_created': 1152,
         'Service_closure_Request_with_network_responsibility': 801,
         'Authorization_Requested': 387,
         'Service_closure_Request_with_BO_responsibility': 638,
         'Pending_Request_for_Reservation_Closure': 399,
         'Pending_Liquidation_Request': 215,
         'Evaluating_Request_(NO_registered_letter)': 405,
         'Pending_Request_for_Network_Information': 28,
         'Back-Office_Adjustment_Requested': 16,
         'Network_Adjustment_Requested': 87,
         'Pending_Request_for_acquittance_of_heirs': 18,
         'Evaluating_Request_(WITH_registered_letter)': 40})

In [406]:
import torch.nn.functional as F

In [419]:
a = F.softmax(torch.tensor([[float('-inf'),float('-inf')],[9,9]]), dim=-1)
a[a.isnan()]=0
a

tensor([[0.0000, 0.0000],
        [0.5000, 0.5000]])

In [70]:
counter = Counter()
# counter.update([each for each in data_add_prev['prev_acts']])


In [71]:
counter

Counter({'Request_created': 1152,
         'Service_closure_Request_with_network_responsibility': 801,
         'Authorization_Requested': 387,
         'Service_closure_Request_with_BO_responsibility': 638,
         'Pending_Request_for_Reservation_Closure': 399,
         'Pending_Liquidation_Request': 215,
         'Evaluating_Request_(NO_registered_letter)': 405,
         'Pending_Request_for_Network_Information': 28,
         'Back-Office_Adjustment_Requested': 16,
         'Network_Adjustment_Requested': 87,
         'Pending_Request_for_acquittance_of_heirs': 18,
         'Evaluating_Request_(WITH_registered_letter)': 40})